In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv('diabetes.csv')

In [29]:
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [30]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [31]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [32]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [33]:
X = scaler.fit_transform(X)

In [34]:
X.shape

(768, 8)

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

In [36]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense


In [37]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4917 - loss: 0.7381 - val_accuracy: 0.5974 - val_loss: 0.6650
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5346 - loss: 0.6868 - val_accuracy: 0.6558 - val_loss: 0.6175
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6336 - loss: 0.6262 - val_accuracy: 0.7532 - val_loss: 0.5830
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6832 - loss: 0.6022 - val_accuracy: 0.7792 - val_loss: 0.5560
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7154 - loss: 0.5842 - val_accuracy: 0.7857 - val_loss: 0.5356
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7190 - loss: 0.5708 - val_accuracy: 0.7922 - val_loss: 0.5179
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7486 - loss: 0.5347 - val_accuracy: 0.8117 - val_loss: 0.5044
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7658 - loss: 0.5224 - val_accuracy: 0.8182 - v

In [40]:
# 1. How to select appropriate optimizer
# 2. No. of nodes or neuron in a layer
# 3. How to select no. of layers
# 4. All in all one model

In [41]:
import kerastuner as kt

In [42]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer', values = ['rmsprop', 'sgd', 'adam', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [43]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy',
                        max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [44]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [45]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [46]:
model = tuner.get_best_models(num_models=1)[0]

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [47]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=[X_test, y_test])

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7371 - loss: 0.5638 - val_accuracy: 0.7727 - val_loss: 0.5380
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7232 - loss: 0.5672 - val_accuracy: 0.7792 - val_loss: 0.5194
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7616 - loss: 0.5170 - val_accuracy: 0.7597 - val_loss: 0.5063
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7378 - loss: 0.5158 - val_accuracy: 0.7597 - val_loss: 0.4996
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7637 - loss: 0.5084 - val_accuracy: 0.7662 - val_loss: 0.4920
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7483 - loss: 0.5108 - val_accuracy: 0.7597 - val_loss: 0.4862
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7500 - loss: 0.5267 - val_accuracy: 0.7597 - val_loss: 0.4832
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7343 - loss: 0.5268 - val_accuracy: 0.746

## Find No of Neuron

In [54]:
def build_model(hp):
    model = Sequential()
    
    unints = hp.Int('units', min_value = 8,max_value = 128, step = 8)    

    model.add(Dense(units=unints, activation='relu', input_dim = 8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [55]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy',
                        max_trials=10,
                        directory = 'mydir',
                        project_name = 'hp_tuning'
                        )

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 10 Complete [00h 00m 01s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 15s


In [57]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [59]:
model = tuner.get_best_models(num_models=1)[0]

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [60]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7443 - loss: 0.5003 - val_accuracy: 0.8052 - val_loss: 0.4620
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7846 - loss: 0.4449 - val_accuracy: 0.7987 - val_loss: 0.4608
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8001 - loss: 0.4394 - val_accuracy: 0.7987 - val_loss: 0.4601
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7853 - loss: 0.4429 - val_accuracy: 0.8052 - val_loss: 0.4591
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7662 - loss: 0.4566 - val_accuracy: 0.7987 - val_loss: 0.4595
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8014 - loss: 0.4333 - val_accuracy: 0.7987 - val_loss: 0.4573
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7857 - loss: 0.4466 - val_accuracy: 0.7987 - val_loss: 0.4584
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7623 - loss: 0.4695 - val_accuracy: 0.79